In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive

/content/drive


In [ ]:
ls

MyDrive/


In [ ]:
cd MyDrive/

/content/drive/MyDrive


In [ ]:
ls

 2211.02001v1.pdf           'coop presentation'/   GenAI.jpg                         Resume/
'Colab Notebooks'/           Deepcase/            'NOC Presentation -Vikrant.pptx'  'Tax Docs'/
'consent form04242024.pdf'   DNRTI/               'Research Papers'/


In [ ]:
cd DNRTI

/content/drive/MyDrive/DNRTI


In [ ]:
ls


test.txt  train.txt  Untitled0.ipynb  valid.txt


In [ ]:
import pandas as pd
import json

In [ ]:
def data_formatter(file_path):
  # Read file...
  file = open(file_path, "r")
  data = file.readlines()

  # Get all tokens with their corresponding labels...
  sentences = []
  sentence = []
  for i in range(len(data)):
    if data[i] != '\n':
      sentence.append(data[i])
    else:
      sentences.append(sentence)
      sentence = []

  # Divide the sentences into actual tokens list and labels list...
  labels = []
  sents = []
  for i in range(len(sentences)):
    sentence_list = []
    label_list = []
    for j in range(len(sentences[i])):
      splits = sentences[i][j].split()
      if len(splits) > 2: # Considering the last word of the token sequence is significant...
        sentence, label = splits[-2], splits[-1]
        continue
      if len(splits) < 2: # Blank Line...
        # print("i = ", i, " j = ", j)
        continue
      sentence, label = splits[0], splits[1]
      sentence_list.append(sentence)

      # Merge 'Idus' with 'Org', and 'Tool' with 'SamFile' to reduce the number of classes from 27 to 23...
      if 'Idus' in label:
        label = label.replace('Idus', 'Org')
      elif 'Tool' in label:
        label = label.replace('Tool', 'SamFile')
      label_list.append(label)
    labels.append(label_list)
    sents.append(sentence_list)

  # Get the unique class names...
  unique_classes = list(set(i for j in labels for i in j))

  # Encode the labels from text to numbers...
  for i in range(len(labels)):
    label_assigned = False
    for k in range(len(labels[i])):
      for j in range(len(unique_classes)):
        if labels[i][k] == unique_classes[j]:
          labels[i][k] = j
          label_assigned = True
          break
      if label_assigned:
        continue

  # Prepare for JSON export...
  json_export = []
  for i in range(len(sents)):
    json_export.append({"tokens": sents[i], "ner_tags": labels[i]})

  return unique_classes, json_export

In [ ]:
# Export to JSON file
def export2JSON(data, export_file_path):
  with open(export_file_path, 'w') as f:
    for d in data:
      json.dump(d, f)
      f.write('\n')

In [ ]:
train_unique_classes, train_json = data_formatter("train.txt")
export2JSON(train_json, 'train_json.jsonl')

In [ ]:
valid_unique_classes, valid_json = data_formatter("valid.txt")
export2JSON(valid_json, 'valid_json.jsonl')

In [ ]:
test_unique_classes, test_json = data_formatter("test.txt")
export2JSON(test_json, 'test_json.jsonl')

In [ ]:
train_unique_classes

['I-Org',
 'I-SecTeam',
 'O',
 'B-Exp',
 'I-Purp',
 'B-Purp',
 'I-Features',
 'I-Time',
 'B-SecTeam',
 'B-SamFile',
 'B-Area',
 'I-Area',
 'B-HackOrg',
 'B-Way',
 'B-OffAct',
 'B-Org',
 'I-Exp',
 'I-OffAct',
 'B-Features',
 'B-Time',
 'I-SamFile',
 'I-Way',
 'I-HackOrg']